## Collecte de données

Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :

1.  Créer un dossier appelé *images*.
2.  Télécharger les images sous licence ouverte dans le dossier *images* (minimum 100
    images).
3.  Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image (.jpeg,
    .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.),
    date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations [Exif](https://en.wikipedia.org/wiki/Exif) présentes dans les fichiers d'images.

In [1]:
import os

# Spécifiez le chemin du dossier que vous souhaitez créer
dossier = 'images'

# Créez le dossier
os.makedirs(dossier, exist_ok=True)

print(f"Le dossier '{dossier}' a été créé avec succès.")


Le dossier 'images' a été créé avec succès.


In [ ]:
! pip install SPARQLWrapper

In [ ]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Initialiser le wrapper SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
IMAGE_DIR = "images"  # Dossier de sauvegarde des images

# Définir la requête SPARQL pour obtenir des images de monuments
sparql.setQuery("""
SELECT ?monument ?monumentLabel ?image WHERE {
  ?monument wdt:P31 wd:Q839954.  # Monument
  ?monument wdt:P18 ?image.      # Image
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)

try:
    # Exécuter la requête et récupérer les résultats en JSON
    results = sparql.query().convert()
except Exception as e:
    print(f"❌ Erreur lors de l'exécution de la requête SPARQL : {e}")
    exit(1)

# Liste des images à télécharger avec des indices de 1 à 100
images = [(i + 1, result["image"]["value"]) for i, result in enumerate(results["results"]["bindings"])]

# Définition d'un User-Agent personnalisé
HEADERS = {
    "User-Agent": "MonScript/1.0 (mailto:mohamedguef@gmail.com)"  # Remplace par ton email pour respecter la politique
}

def download_image(data):
    index, image_url = data
    try:
        # Nom fixe sous le format image_1.jpg, image_2.jpg, ..., image_100.jpg
        image_name = f"image_{index}.jpg"
        image_path = os.path.join(IMAGE_DIR, image_name)

        # Télécharger l'image avec un User-Agent correct
        response = requests.get(image_url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Vérifie si la requête a réussi
        
        # Sauvegarder l'image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ Téléchargé : {image_name}")

    except requests.RequestException as e:
        print(f"⚠️ Erreur lors du téléchargement de {image_url} : {e}")
    except Exception as e:
        print(f"⚠️ Erreur inattendue : {e}")

# Téléchargement des images en parallèle
MAX_THREADS = 10  # Ajuste ce nombre en fonction de ta connexion et des performances

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, images)

print("✅ Tous les téléchargements sont terminés !")


✅ Téléchargé : image_4.jpg
✅ Téléchargé : image_9.jpg
✅ Téléchargé : image_10.jpg
✅ Téléchargé : image_3.jpg
✅ Téléchargé : image_2.jpg
✅ Téléchargé : image_8.jpg
✅ Téléchargé : image_7.jpg
✅ Téléchargé : image_5.jpg
✅ Téléchargé : image_12.jpg
✅ Téléchargé : image_1.jpg
✅ Téléchargé : image_11.jpg
✅ Téléchargé : image_14.jpg
✅ Téléchargé : image_13.jpg
✅ Téléchargé : image_6.jpg
✅ Téléchargé : image_15.jpg
✅ Téléchargé : image_16.jpg
✅ Téléchargé : image_18.jpg
✅ Téléchargé : image_21.jpg
✅ Téléchargé : image_22.jpg
✅ Téléchargé : image_19.jpg
✅ Téléchargé : image_25.jpg
✅ Téléchargé : image_24.jpg
✅ Téléchargé : image_27.jpg
✅ Téléchargé : image_20.jpg
✅ Téléchargé : image_28.jpg
✅ Téléchargé : image_30.jpg
✅ Téléchargé : image_31.jpg
✅ Téléchargé : image_32.jpg
✅ Téléchargé : image_33.jpg
✅ Téléchargé : image_29.jpg
✅ Téléchargé : image_34.jpg
✅ Téléchargé : image_36.jpg
✅ Téléchargé : image_37.jpg
✅ Téléchargé : image_41.jpg
✅ Téléchargé : image_40.jpg
✅ Téléchargé : image_26.jpg
✅

In [ ]:
! pip install Pillow ExifRead

In [ ]:
! pip install tqdm

In [13]:
import os
import json
from PIL import Image, UnidentifiedImageError
import exifread
from tqdm import tqdm  # Barre de progression

# Définir le dossier des images et le fichier de sortie JSON
IMAGE_DIR = "images"
METADATA_FILE = os.path.join(IMAGE_DIR, "metadata.json")

# Vérifier que le dossier "images" existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable. Vérifiez que les images sont bien téléchargées.")
    exit(1)

# Liste pour stocker les métadonnées de toutes les images
metadata_list = []

# Liste des fichiers images triés
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Parcourir les images et extraire les métadonnées
for image_name in tqdm(image_files, desc="📷 Extraction des métadonnées"):
    image_path = os.path.join(IMAGE_DIR, image_name)

    try:
        # Vérifier si l'image est corrompue en tentant de l'ouvrir
        with Image.open(image_path) as img:
            img.verify()  # Vérifie l'intégrité de l'image sans la charger
            img = Image.open(image_path)  # Recharge l'image pour la lecture des métadonnées
            width, height = img.size
            format_ = img.format
            orientation = 'Portrait' if height > width else 'Paysage' if width > height else 'Carré'

        # Vérifier si c'est un PNG (pas d'Exif)
        if format_.upper() == "PNG":
            exif_data = "Non disponible (format PNG)"
        else:
            # Extraire les métadonnées Exif avec exifread
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal", details=False)

            # Filtrer les champs problématiques
            exif_data = {
                "Date de création": str(tags.get("EXIF DateTimeOriginal", "Inconnu")),
                "Modèle d'appareil": str(tags.get("Image Model", "Inconnu")),
                "Marque d'appareil": str(tags.get("Image Make", "Inconnu")),
                "ISO": str(tags.get("EXIF ISOSpeedRatings", "Inconnu")),
                "Temps d'exposition": str(tags.get("EXIF ExposureTime", "Inconnu")),
                "Ouverture (f)": str(tags.get("EXIF FNumber", "Inconnu")),
                "Longueur focale": str(tags.get("EXIF FocalLength", "Inconnu")),
                "GPS Latitude": str(tags.get("GPS GPSLatitude", "Non disponible")),
                "GPS Longitude": str(tags.get("GPS GPSLongitude", "Non disponible")),
            }

            # Supprimer les champs corrompus (évite les erreurs "Possibly corrupted field")
            exif_data = {key: value for key, value in exif_data.items() if "Possibly corrupted" not in value}

        # Ajouter les métadonnées à la liste
        metadata_list.append({
            'Nom du fichier': image_name,
            'Taille': {'Largeur': width, 'Hauteur': height},
            'Format': format_,
            'Orientation': orientation,
            'Exif': exif_data
        })

    except UnidentifiedImageError:
        print(f"⚠️ Image illisible : {image_name} (corrompue ou format inconnu)")
    except Exception as e:
        print(f"⚠️ Erreur inattendue sur {image_name} : {e}")

# Enregistrer toutes les métadonnées dans un fichier JSON
try:
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"✅ Métadonnées enregistrées dans '{METADATA_FILE}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")


📷 Extraction des métadonnées: 100%|██████████| 100/100 [00:01<00:00, 58.15it/s]

✅ Métadonnées enregistrées dans 'images\metadata.json'


In [ ]:
! pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
! pip install scikit-learn

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.9/11.1 MB 16.8 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 21.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 20.5 MB/s eta 0:00:00


In [14]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import json

# Fonction pour extraire les couleurs dominantes
def get_dominant_colors(image_path, k=3):
    """
    Extrait les k couleurs dominantes d'une image en utilisant K-Means.
    :param image_path: Chemin de l'image
    :param k: Nombre de couleurs dominantes à extraire
    :return: Liste des couleurs dominantes au format RGB
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Impossible de lire l'image : {image_path}")
        return None
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB
    pixels = image.reshape(-1, 3)  # Redimensionner en tableau 2D (N pixels x 3 canaux)
    kmeans = KMeans(n_clusters=k)  # Appliquer K-Means
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)  # Extraire les couleurs dominantes
    return colors.tolist()  # Convertir en liste pour JSON

# Dossier contenant les images
IMAGE_DIR = "images"

# Vérifier que le dossier existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable.")
    exit(1)

# Dictionnaire pour stocker les couleurs dominantes de chaque image
dominant_colors_data = {}

# Parcourir toutes les images du dossier
for image_name in os.listdir(IMAGE_DIR):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers images
        image_path = os.path.join(IMAGE_DIR, image_name)
        print(f"🔍 Traitement de l'image : {image_name}")

        # Extraire les couleurs dominantes
        dominant_colors = get_dominant_colors(image_path, k=3)  # Extraire 3 couleurs dominantes
        if dominant_colors:
            dominant_colors_data[image_name] = dominant_colors
            print(f"✅ Couleurs dominantes pour {image_name} : {dominant_colors}")
        else:
            print(f"⚠️ Aucune couleur dominante trouvée pour {image_name}")

# Enregistrer les résultats dans un fichier JSON
output_file = os.path.join(IMAGE_DIR, "dominant_colors.json")
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dominant_colors_data, f, indent=4, ensure_ascii=False)
    print(f"✅ Couleurs dominantes enregistrées dans '{output_file}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")

🔍 Traitement de l'image : image_1.jpg
✅ Couleurs dominantes pour image_1.jpg : [[165, 203, 247], [48, 43, 35], [169, 136, 91]]
🔍 Traitement de l'image : image_10.jpg
✅ Couleurs dominantes pour image_10.jpg : [[182, 179, 172], [76, 66, 55], [226, 224, 218]]
🔍 Traitement de l'image : image_100.jpg
✅ Couleurs dominantes pour image_100.jpg : [[173, 128, 90], [188, 191, 202], [77, 44, 25]]
🔍 Traitement de l'image : image_11.jpg
✅ Couleurs dominantes pour image_11.jpg : [[135, 132, 119], [187, 186, 169], [73, 69, 64]]
🔍 Traitement de l'image : image_12.jpg
✅ Couleurs dominantes pour image_12.jpg : [[62, 54, 43], [189, 215, 246], [143, 111, 94]]
🔍 Traitement de l'image : image_13.jpg
✅ Couleurs dominantes pour image_13.jpg : [[197, 215, 234], [65, 49, 23], [136, 110, 77]]
🔍 Traitement de l'image : image_14.jpg
✅ Couleurs dominantes pour image_14.jpg : [[225, 221, 207], [42, 37, 29], [143, 139, 126]]
🔍 Traitement de l'image : image_15.jpg
✅ Couleurs dominantes pour image_15.jpg : [[75, 75, 75]

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


✅ Couleurs dominantes pour image_80.jpg : [[255, 255, 255], [0, 0, 0], [255, 255, 255]]
🔍 Traitement de l'image : image_81.jpg
✅ Couleurs dominantes pour image_81.jpg : [[71, 108, 175], [63, 65, 46], [165, 154, 131]]
🔍 Traitement de l'image : image_82.jpg
✅ Couleurs dominantes pour image_82.jpg : [[49, 61, 27], [135, 187, 232], [116, 127, 65]]
🔍 Traitement de l'image : image_83.jpg
✅ Couleurs dominantes pour image_83.jpg : [[54, 62, 47], [143, 134, 98], [159, 205, 240]]
🔍 Traitement de l'image : image_84.jpg
✅ Couleurs dominantes pour image_84.jpg : [[185, 179, 172], [129, 119, 103], [61, 59, 49]]
🔍 Traitement de l'image : image_85.jpg
✅ Couleurs dominantes pour image_85.jpg : [[67, 58, 53], [219, 224, 222], [182, 177, 183]]
🔍 Traitement de l'image : image_86.jpg
✅ Couleurs dominantes pour image_86.jpg : [[171, 202, 231], [40, 47, 42], [110, 118, 117]]
🔍 Traitement de l'image : image_87.jpg
✅ Couleurs dominantes pour image_87.jpg : [[122, 116, 75], [195, 184, 160], [55, 53, 23]]
🔍 Trai

In [2]:
! pip install tensorflow

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl (390.2 MB)


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import json
from PIL import UnidentifiedImageError

# Charger le modèle ResNet50 pré-entraîné
model = ResNet50(weights='imagenet')

def classify_image(image_path):
    try:
        # Charger et prétraiter l'image
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Prédire les classes
        preds = model.predict(x)
        return decode_predictions(preds, top=3)[0]

    except UnidentifiedImageError:
        print(f"Erreur : Le fichier {image_path} n'est pas une image valide.")
        return []

# Liste pour stocker les prédictions de toutes les images
predictions_list = []

# Parcourir toutes les images dans le dossier 'images'
for file_name in os.listdir('images'):
    if file_name.startswith('image_') and file_name.endswith('.jpg'):
        image_path = os.path.join('images', file_name)

        # Classifier l'image
        predictions = classify_image(image_path)

        # Ajouter les prédictions à la liste si l'image est valide
        if predictions:
            predictions_list.append({
                'file_name': file_name,
                'predictions': [{'label': pred[1], 'score': float(pred[2])} for pred in predictions]
            })

        print(f"Prédictions pour {file_name}: {predictions}")

# Enregistrer toutes les prédictions dans un fichier JSON
with open('images/predictions.json', 'w') as f:
    json.dump(predictions_list, f, indent=4)

print("Prédictions enregistrées dans 'images/predictions.json'")


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
Prédictions pour image_1.jpg: [('n02980441', 'castle', 0.60829854), ('n03781244', 'monastery', 0.13541363), ('n03028079', 'church', 0.0858401)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Prédictions pour image_10.jpg: [('n03743016', 'megalith', 0.62844753), ('n02894605', 'breakwater', 0.22555366), ('n09428293', 'seashore', 0.03147549)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Prédictions pour image_100.jpg: [('n04326547', 'stone_wall', 0.3791062), ('n03743016', 'megalith', 0.3281441), ('n01687978', 'agama', 0.034389164)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Prédictions pour image_11.jpg: [('n03804744', 'nail', 0.42812166), ('n02219486', 'ant', 0.10862584), ('n01737021', 'water_snake', 0.033621844)]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Prédictions pour image_12.jpg: [('n03028079', 'church', 0.7359976), ('n02825657', 'bell_cote', 0.2626674), ('n02814860', 'beaco